In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
os.chdir("/home/573/rh2942/WASSA-2023-EMP") # changing dir for evaluation file
# print(os.getcwd())

In [3]:
raw_train = pd.read_csv('./dataset/WASSA23_essay_level_with_labels_train.tsv', sep='\t', na_values='unknown', header=0) # raw csv file consists of 'unknown' values
raw_train.sample(2)

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,speaker_number,split,essay_id,emotion
601,268,47,The whole article broke my heart. If I got a t...,6.833333,6.625,30,1.0,6.0,3.0,37.0,...,6.5,6.0,4.857,2.000,3.429,5.000,2,train,767,Sadness
19,32,386,The fact that we don't take care of our vetera...,6.000000,5.250,44,2.0,6.0,1.0,46.0,...,5.0,5.0,3.286,2.286,4.857,4.714,1,train,31,Anger


In [6]:
train_dev = pd.read_csv("./processed_data/essay_article_id_train_dev.csv", index_col=0)
train_dev.sample(2)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,...,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
492,148,147,Check this out....haze from the fires in Indon...,32,2.0,2.0,1.0,34.0,50000.0,2,...,Surprise,6.5,3.5,1.0,6.0,7.0,2.143,1.000,1.571,2.286
572,246,177,Eh I don't have strong feelings about this. I...,24,2.0,7.0,1.0,38.0,42000.0,2,...,Sadness,7.0,3.5,6.5,5.5,6.5,3.429,2.714,2.571,3.857


In [4]:
article_raw = pd.read_csv('./dataset/articles_adobe_AMT.csv', header=0, index_col=0)
article_raw.sample(2)

,text
article_id,
90,Chicago just shut down hundreds of public drin...
219,Missing Australian hiker found after 2 weeks i...


In [6]:
raw_dev = pd.read_csv('./dataset/dev/WASSA23_essay_level_dev.tsv', sep='\t', header=0)
raw_dev.sample(2)

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,split,essay_id
22,84,353,So I just read an article about how a father a...,63,1,4,1,25,29000,1,dev,83
58,220,247,The terrific massacre that happened a year ago...,82,1,7,1,46,150000,1,dev,219


In [3]:
WS23_WS22 = pd.read_csv("./processed_data/train_dev_WS23_train_WS22.csv", index_col=0)
WS23_WS22

,article_id,essay,gender,education,race,age,income,empathy,distress,emotion,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern
0,35,It breaks my heart to see people living in tho...,1.0,6.0,3.0,37.0,40000.0,6.833333,6.625,Hope/Sadness,7.00,5.50,1.00,6.50,6.0,4.857,2.0000,3.429,5.000
1,35,I wonder why there aren't more people trying t...,1.0,6.0,2.0,32.0,35000.0,5.833333,6.000,Anger,5.50,5.00,2.00,5.50,4.5,3.429,2.8570,2.857,2.714
2,35,"After reading the article, you can't help but ...",1.0,6.0,1.0,29.0,85000.0,1.000000,1.375,Sadness,6.75,6.75,6.75,6.75,7.0,4.643,2.0715,4.143,4.643
3,213,It is so sad that someone who had such an amaz...,2.0,5.0,1.0,28.0,50000.0,6.166667,6.625,Sadness,6.00,6.00,5.00,4.50,3.5,5.000,4.1430,4.857,5.000
4,213,"From reading the article, it looks like the wo...",1.0,6.0,3.0,37.0,40000.0,6.833333,1.000,Neutral,7.00,5.50,1.00,6.50,6.0,4.857,2.0000,3.429,5.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2842,16,"These days, women are winning in sports. There...",2.0,4.0,3.0,34.0,30000.0,1.000000,1.000,joy,7.00,7.00,7.00,7.00,7.0,5.000,1.2860,4.286,4.429
2843,158,I hate ISIS. They are a group full of hate and...,2.0,4.0,3.0,34.0,30000.0,1.000000,1.000,anger,7.00,7.00,7.00,7.00,7.0,5.000,1.2860,4.286,4.429
2844,203,This is just disgusting. I cannot believe that...,2.0,4.0,3.0,34.0,30000.0,1.000000,1.000,disgust,7.00,7.00,7.00,7.00,7.0,5.000,1.2860,4.286,4.429
2845,253,I feel like the world is so corrupt that it no...,2.0,4.0,3.0,34.0,30000.0,1.000000,1.000,sadness,7.00,7.00,7.00,7.00,7.0,5.000,1.2860,4.286,4.429


In [ ]:
# raw_test = pd.read_csv('./dataset/dev/', sep='\t', header=0)
# raw_test.sample(3)

In [7]:
def num_to_text(raw_data):
    input_data = raw_data.copy() #mandatory step as dataframe is mutable
    
    input_data.dropna(inplace=True)
    print(f"Existing non-numeric columns:\n {input_data.select_dtypes(exclude=['number']).columns.tolist()}") #Just checking if any non-numeric value exist on numeric columns
    
    #converting article id to corresponding article texts
    input_data['article'] = input_data['article_id'].apply(lambda x: article_raw.loc[x, 'text'])
    
    categories = {1.: "I am male.", 2.: "I am female.", 5: "I am nonbinary."}
    input_data['gender'] = input_data['gender'].map(categories)

    categories = {
        1.: "My education level is Less than a high school diploma.",
        2.: "My education level is High School diploma.",
        3.: "My education level is echnical/Vocational School.",
        4.: "My education level is Some college but no degree.",
        5.: "My education level is Two year associate degree.",
        6.: "My education level is Four year bachelor’s degree.",
        7.: "My education level is Postgradute or professional degree."
    }
    input_data['education'] = input_data['education'].map(categories)

    categories = {
        1.: "My race is White.",
        2.: "My race is Hispanic or Latino.",
        3.: "My race is Black or African American.",
        4.: "My race is Native American or American Indian.",
        5.: "My race is Asian/Pacific Islander.",
        6.: "My race is other."
    }
    input_data['race'] = input_data['race'].map(categories)

    input_data['age'] = input_data['age'].apply(lambda x: "My age is "+str(x)+" years.")
    input_data['income'] = input_data['income'].apply(lambda x: "My income is "+str(x)+".")
    
    input_data['article_id_text'] = input_data['article_id'].apply(lambda x: "I am writing with respect to article number "+str(x)+".")
    
    # print(input_data.isna().any())
    assert input_data.isna().any().any() == False #no NA values
    assert input_data.isnull().any().any() == False #no null values
    
    input_data['demographic'] = input_data['gender'] + ' ' + input_data['age'] + ' ' + input_data['education'] + ' ' + input_data['race'] + ' ' + input_data['income']  + ' ' + input_data['article_id_text']  
    input_data['demographic_essay'] = input_data['demographic'] + ' ' + input_data['essay']
    
    return input_data

In [8]:
def save_preprocessed(df, dataname):
    raw_data = df.copy()
    processed_df = num_to_text(raw_data)
    processed_df.to_csv("./processed_data/preprocessed_" + dataname + ".csv")

In [9]:
# save_preprocessed(raw_train, dataname="train")
save_preprocessed(train_dev, dataname="train_dev")
# save_preprocessed(raw_dev, dataname="dev")
# save_preprocessed(WS23_WS22, dataname="train_dev_WS23_train_WS22")
# save_preprocessed(raw_test, dataname="test")

Existing non-numeric columns:
 ['essay', 'split', 'emotion']


# OLD

In [38]:
essay_empathy_raw = pd.read_csv('./dataset/WASSA23_essay_level_with_labels_train.tsv', sep='\t', header=0)
article_raw = pd.read_csv('./dataset/articles_adobe_AMT.csv', header=0, index_col=0)

In [39]:
essay_empathy_raw

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,speaker_number,split,essay_id,idx,emotion
0,2,35,It breaks my heart to see people living in tho...,6.714286,6.714286,30,1,6,3,37,...,6.0,4.857,2.0,3.429,5.0,1,train,1,35_2_person1_1,Hope/Sadness
1,3,35,I wonder why there aren't more people trying t...,5.857143,6.000000,19,1,6,2,32,...,4.5,3.429,2.857,2.857,2.714,1,train,2,35_3_person1_2,Anger
2,5,35,"After reading the article, you can't help but ...",1.000000,1.428571,17,1,6,1,29,...,7.0,4.643,2.0715,4.143,4.643,1,train,4,35_5_person1_4,Sadness
3,6,213,It is so sad that someone who had such an amaz...,6.000000,6.857143,16,2,5,1,28,...,3.5,5.0,4.143,4.857,5.0,1,train,5,213_6_person1_5,Sadness
4,8,213,"From reading the article, it looks like the wo...",6.000000,1.000000,30,1,6,3,37,...,6.0,4.857,2.0,3.429,5.0,1,train,7,213_8_person1_7,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,495,218,I feel that this will become a national proble...,6.571429,6.714286,30,1,6,3,37,...,6.0,4.857,2.0,3.429,5.0,2,train,994,218_495_person2_494,Neutral
788,496,103,The whole situation is sketchy. The wavering r...,3.571429,6.428571,16,2,5,1,28,...,3.5,5.0,4.143,4.857,5.0,2,train,995,103_496_person2_495,Neutral
789,498,103,The death of a former aide to Russian Presiden...,5.428571,2.000000,43,2,6,1,33,...,7.0,4.286,1.286,3.857,4.0,2,train,997,103_498_person2_497,Neutral
790,499,103,Everything about Russia really freaks me out. ...,6.000000,6.000000,53,2,3,1,27,...,6.0,4.0,2.286,3.571,3.714,2,train,998,103_499_person2_498,Fear


In [9]:
len(essay_empathy_raw.iloc[0,2])

414

In [7]:
len(article_raw.iloc[0,0])

2396

In [5]:
# essay_empathy_raw.columns

In [10]:
essay_empathy_raw.dtypes

conversation_id                    int64
article_id                         int64
essay                             object
empathy                          float64
distress                         float64
speaker_id                         int64
gender                            object
education                         object
race                              object
age                               object
income                            object
personality_conscientiousness     object
personality_openess               object
personality_extraversion          object
personality_agreeableness         object
personality_stability             object
iri_perspective_taking            object
iri_personal_distress             object
iri_fantasy                       object
iri_empathatic_concern            object
speaker_number                     int64
split                             object
essay_id                           int64
idx                               object
emotion         

In [22]:
essay_empathy = essay_empathy_raw.copy()

In [13]:
essay_empathy['speaker_id'].unique()

array([30, 19, 17, 16, 49, 24, 43, 31, 40, 13, 37, 20, 44, 45, 33, 48, 25,
       56, 57, 23,  5, 27, 55,  2,  7, 53, 35, 32, 18, 14, 46, 51, 22, 36,
       15, 41,  3, 50, 38,  8, 26,  6, 34, 52,  9, 42,  0, 58, 10, 12, 29,
       47, 11,  1,  4, 28, 39, 21, 54])

In [ ]:
essay_empathy.drop(columns=['speaker_id','split','essay_id','idx'], inplace=True)

In [ ]:
essay_empathy.head()

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,speaker_number,idx,emotion
0,2,35,It breaks my heart to see people living in tho...,6.714286,6.714286,30,1,6,3,37,...,1.0,6.5,6.0,4.857,2.0,3.429,5.0,1,35_2_person1_1,Hope/Sadness
1,3,35,I wonder why there aren't more people trying t...,5.857143,6.000000,19,1,6,2,32,...,2.0,5.5,4.5,3.429,2.857,2.857,2.714,1,35_3_person1_2,Anger
2,5,35,"After reading the article, you can't help but ...",1.000000,1.428571,17,1,6,1,29,...,6.75,6.75,7.0,4.643,2.0715,4.143,4.643,1,35_5_person1_4,Sadness
3,6,213,It is so sad that someone who had such an amaz...,6.000000,6.857143,16,2,5,1,28,...,5.0,4.5,3.5,5.0,4.143,4.857,5.0,1,213_6_person1_5,Sadness
4,8,213,"From reading the article, it looks like the wo...",6.000000,1.000000,30,1,6,3,37,...,1.0,6.5,6.0,4.857,2.0,3.429,5.0,1,213_8_person1_7,Neutral


In [41]:
all_columns = list(essay_empathy.columns)
# all_columns

In [42]:
string_columns = ['essay', 'emotion']
numeric_columns = [item for item in all_columns if item not in string_columns] # all other columns
# numeric_columns

In [43]:
essay_empathy = (essay_empathy.drop(columns=numeric_columns).join(essay_empathy[numeric_columns].apply(pd.to_numeric, errors='coerce'))) #converting non-numeric to null
essay_empathy.dropna(axis=0, inplace=True) #removing the rows having NA values
# essay_empathy

In [44]:
essay_empathy.isnull().any(axis=0)

essay                            False
emotion                          False
idx                              False
conversation_id                  False
article_id                       False
empathy                          False
distress                         False
speaker_id                       False
gender                           False
education                        False
race                             False
age                              False
income                           False
personality_conscientiousness    False
personality_openess              False
personality_extraversion         False
personality_agreeableness        False
personality_stability            False
iri_perspective_taking           False
iri_personal_distress            False
iri_fantasy                      False
iri_empathatic_concern           False
speaker_number                   False
dtype: bool

In [45]:
essay_empathy

,essay,emotion,idx,conversation_id,article_id,empathy,distress,speaker_id,gender,education,...,personality_conscientiousness,personality_openess,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,speaker_number


In [14]:
# essay_empathy['emotion'].nunique()

In [15]:
essay_empathy['emotion'] = pd.Categorical(essay_empathy['emotion']).codes   #convert emotion texts to numbers
# essay_empathy.head()

In [16]:
essay_empathy.dtypes

essay                             object
emotion                             int8
conversation_id                    int64
article_id                         int64
empathy                          float64
distress                         float64
gender                           float64
education                        float64
race                             float64
age                              float64
income                           float64
personality_conscientiousness    float64
personality_openess              float64
personality_extraversion         float64
personality_agreeableness        float64
personality_stability            float64
iri_perspective_taking           float64
iri_personal_distress            float64
iri_fantasy                      float64
iri_empathatic_concern           float64
speaker_number                     int64
dtype: object

In [17]:
essay_empathy['essay'] = essay_empathy['essay'].apply(lambda x: x.replace('\n', ' '))   #converting newlines (if any) to space

In [18]:
essay_empathy.to_csv("./essay_texts_all_preprocessed.csv")

# FastText embeddings

In [17]:
# essay_empathy.head()

In [18]:
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')

In [19]:
essay_fasttext = pd.DataFrame(essay_empathy['essay'].apply(lambda x: ft.get_sentence_vector(x)))
# essay_fasttext

In [20]:
# expanding embedding list to columns
essay_fasttext = pd.concat([essay_fasttext.drop(columns='essay'), pd.DataFrame(essay_fasttext['essay'].tolist(), index=essay_fasttext.index).add_prefix('fasttext_')],axis=1)

In [21]:
# essay_fasttext

In [22]:
essay_empathy.drop(columns='essay', inplace=True) # no longer required

In [23]:
# essay_empathy

In [24]:
empathy_fasttext = pd.concat([essay_empathy, essay_fasttext], axis=1)

In [25]:
empathy_fasttext

,emotion,conversation_id,article_id,empathy,distress,gender,education,race,age,income,...,fasttext_290,fasttext_291,fasttext_292,fasttext_293,fasttext_294,fasttext_295,fasttext_296,fasttext_297,fasttext_298,fasttext_299
0,20,2,35,6.714286,6.714286,1.0,6.0,3.0,37.0,40000.0,...,0.031908,0.008618,-0.084222,0.009917,-0.015905,0.013034,0.009217,0.101176,-0.013040,0.002871
1,0,3,35,5.857143,6.000000,1.0,6.0,2.0,32.0,35000.0,...,0.036240,0.009713,-0.074727,-0.008554,-0.012608,0.013154,0.003391,0.072847,-0.003598,-0.001064
2,27,5,35,1.000000,1.428571,1.0,6.0,1.0,29.0,85000.0,...,0.029181,0.004013,-0.071510,-0.003602,-0.014522,0.005235,-0.002368,0.090139,-0.004795,0.012187
3,27,6,213,6.000000,6.857143,2.0,5.0,1.0,28.0,50000.0,...,0.033555,0.014324,-0.072501,0.023933,-0.021073,0.010637,-0.005696,0.088332,-0.022485,0.023060
4,24,8,213,6.000000,1.000000,1.0,6.0,3.0,37.0,40000.0,...,0.028484,-0.005003,-0.073540,0.002448,-0.019296,0.008079,0.011888,0.088122,-0.007650,-0.003173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,24,495,218,6.571429,6.714286,1.0,6.0,3.0,37.0,40000.0,...,0.016001,-0.012279,-0.068207,0.004699,-0.006312,-0.000172,0.004574,0.092242,-0.002821,0.023676
788,24,496,103,3.571429,6.428571,2.0,5.0,1.0,28.0,50000.0,...,0.044965,0.000671,-0.071815,0.007341,-0.009898,0.017540,0.007518,0.066064,-0.006650,0.003420
789,24,498,103,5.428571,2.000000,2.0,6.0,1.0,33.0,110000.0,...,0.016542,0.000684,-0.024153,0.012848,-0.005384,-0.005280,-0.023967,0.063515,0.011698,-0.004720
790,14,499,103,6.000000,6.000000,2.0,3.0,1.0,27.0,25000.0,...,0.038192,-0.000660,-0.081046,-0.002569,-0.011361,0.006441,0.008258,0.089426,-0.011913,0.003074


In [26]:
empathy_fasttext.to_csv('./empathy_fasttext.csv')